In [1]:
import pandas as pd

In [2]:
# set up file directory and run_id

run_dir = ''
run_id = ''

In [3]:
# summarize residential lump sum acctlog

all_data = pd.DataFrame(columns = ['amount', 'year', 'county', 'residential_units',
                                   'subsidized_units', 'inclusionary_units', 'deed_restricted_units'])
subsidized_units = 0
inclusionary_units = 0
deed_restricted_units = 0
residential_units = 0

for county in ['Alameda', 'Contra Costa', 'Marin', 'Napa', 'San Francisco',
               'San Mateo', 'Santa Clara', 'Solano', 'Sonoma']:

    print(county)
    df = pd.read_csv('{}\\run{}_acctlog_{} Affordable Housing Bond_2050.csv'.format(run_dir, run_id, county))

    df['county'] = county

    if 'subsidized_units' in df.columns:
        subsidized = df.subsidized_units.sum()
        print('Number of subsizided units: {:,}'.format(int(subsidized)))
        subsidized_units += subsidized
        
        inclusionary = df.inclusionary_units.sum()
        inclusionary_units += inclusionary

        deed_restricted = df.deed_restricted_units.sum()
        deed_restricted_units += deed_restricted
        
        residential = df.residential_units.sum()
        residential_units += residential
        
        fee_used = (df.loc[df.amount < 0].amount.sum()) * -1
        print('Amount of funding used: {}'.format(fee_used))
        fee_left = df.amount.sum()
        print('Amount of funding left: {}'.format(fee_left))

        first = df.loc[df.residential_units > 0].year.min()
        print('First round a building is subsidized: {}'.format(first))
        print('Subsidized_units built by year: \n{}\n'.format(
            df.loc[df.residential_units > 0].groupby(['year'])['subsidized_units'].sum()))

        all_data = pd.concat([all_data, df])

    else:
        print('No subsidized units.')
        fee_left = df.amount.sum()
        print('Amount of funding left: {} \n'.format(fee_left))
        all_data = pd.concat([all_data, df])
    
print('Total number of subsidized_units: {:,}'.format(int(subsidized_units)))
print('Number of inclusionary_units could not have been created without the subsidy: {:,}'.format(int(inclusionary_units)))
print('Number of residential_units could not have been created without the subsidy: {:,}'.format(int(residential_units)))

In [4]:
# export the aggregated file if needed

all_data.drop(columns = ['Unnamed: 0'], inplace = True)
all_data.to_csv('residential_lump_sum_acct_summary.csv', index=False)